In [2]:
// Import der nötigen Namespaces und Module
open System
open System.Collections.Generic

In [3]:
let generateId =
    let mutable counter = 0
    fun () -> 
        counter <- counter + 1
        counter

## Tokenization ##
Die folgende Zelle enthält die für den Tokenization-Prozess nötigen Definitionen.
Tokenization beschreibt den Vorgang des Parsens der eingegebenen Zeichen in ein Zwischenformat welches das folgende semantische Parsen erleichtert. In diesem Vorgang werden auch Fehler geworfen wenn ungültige Zeichen entdeckt werden.
Im Schritt der Tokenization werden Zahlen bereits zusamengeführt, d.h.:
    
    (Zahl 1) (Zahl 2) (Zahl 3) => (Zahl 123)
    
ebenso werden Identifikatoren (d.h. einzelne Zeichen, welche Buchstaben sind) zusammen geführt:

    (Id a) (Id b) (Id c) => (Id abc)
    

In [4]:
///**Description**
/// Aus dem Input eingelesener, geparster, aber noch nicht interpretierter Token.
///
type Token =
    | Id of string
    | Number of int
    | Function of string
    | OpeningBracket
    | ClosingBracket
    | Hat 
    | Plus 
    | Minus 
    | Times
    | Divided
    | Argument
        
///**Description**
/// Trennzeichen, welche den Beginn eines neuen Token bestimmen.
/// Zusätzlich gibt es noch weitere Zeichen welche den Anfang einer neuen Token
/// anzeigen: Funktionsnamen, Identifier nach Zahl,...
///
let delimiters =
    ['('; ')'; '+'; '-'; '*'; '/']
    
///**Description**
/// List von geparsten Tokens.
///
type TokenStream = TokenStreamContent of Token list    

In [5]:
///**Description**
/// Gibt einen Tokenstream über die Console aus.
///
let printTokenStreamContent (TokenStreamContent tokens) =
    let tokenToString (t: Token) =
        match t with
        | (Id x) -> x
        | (Number n) -> n |> string
        | (Token.Function f) -> f
        | OpeningBracket -> "("
        | ClosingBracket -> ")"
        | Hat -> "^"
        | Plus -> "+"
        | Minus -> "-"
        | Times ->  "×"
        | Divided -> "÷"
        | Token.Argument -> "○"
    let concatted = [ for t in tokens do yield (t |> tokenToString) ] 
                    |> String.concat ""
    printfn "%s" concatted

In [6]:
///**Description**
/// Liste von Funktionen, gegen die geprüft wird.
///
let knownFunctions = 
    [ "sin", Math.Sin; "cos", Math.Cos; "tan", Math.Tan; "atan", Math.Atan; "exp", Math.Exp; "sqrt", Math.Sqrt]
    |> Map.ofList

///**Description**
/// Testet ob der übergebene String als Zahl geparst werden kann.
///
let isNumber (s:string) =
    let couldParse, _ = Int32.TryParse(s)
    couldParse
    
let isCharNumber c =
    Char.IsNumber(c)
    
let isCharLetter c =
    Char.IsLetter(c)

///**Description**
/// Erzeugt aus einem String einen Token.
///
let matchToToken s =
    match s with    
       | "+" -> Plus
       | "-" -> Minus
       | "*" -> Times
       | "/" -> Divided
       | "^" -> Hat
       | "(" -> OpeningBracket
       | ")" -> ClosingBracket
       | num  when isNumber s -> 
           Number (int num)
       | func when knownFunctions |> Map.containsKey s -> 
           Token.Function func
       | _ -> Id s
       
///**Description**
///
///**Parameters**
///  * `s` - String, welcher in Tokens umgewandelt werden soll.
///  * `extractor` - Funktion, welche aus dem Gesamtstring einzelne Teile extrahiert, 
///                  die dann in Tokens umgewandelt werden können.
///
let parseStringAsTokens (extractor: string->(string Option * string Option)) (s: string) =
    let rec parseString toParse tokensFound =
        match (extractor toParse) with
        | (None, _) -> // es gibt keine neuen Strings mehr -> fertig.
            tokensFound
        | (Some part, None) -> 
            (matchToToken part) :: tokensFound
        | (Some part, Some rest) -> 
            parseString rest <| (matchToToken part) :: tokensFound
    parseString s []       

## Ausdrücke ##
Die folgende Zelle beschreibt die Definitionen welche verwendet werden um die Tokens in logische Komponenten aufzuteilen. Bricht man die Möglichkeiten herunter gibt es nur wenige Basistypen aus welchen ein Term bestehen kann. Die Möglichkeiten werden im ''type Term'' festgelegt.

In [7]:
///**Description**
/// Repräsentiert die Verbinung zweier Terme.
///
type TermConnector =
    | Addition
    | Subtraction
    | Multiplication
    | Division
    | Argument
    
///**Description**
/// Platzhalter/mögliche künftige Definition für Funktionen in denen man auch die Ableitungen
/// definieren kann. Muss aber vermutich als Term beschrieben werden.
///
type Function<'a> =
    {
        Name: string;
        Power: int;
        Evaluate: ('a->'a);
    }
    
///**Description**
/// Erzeugt für einen übergebenen Funktionsnamen eine neue Funktions-Instanz.
///
let functionFromNameAndPower (name:string) (power:int) =
    let func = knownFunctions.[name]
    {Name=name; Power=power; Evaluate=func}

///**Description**
/// Erzeugt für einen übergebenen Funktionsnamen eine neue Funktions-Instanz.
///
let functionFromName (name:string) =
    functionFromNameAndPower name 1
    
///**Description**
/// Logische Teile des Ausdrucks.
///
type Term =
    | SubTerm   of (Term list)         // z.B. Klammern
    | Polynom   of (int*string*int)        // 2x^3, 3x, oder -10 (implizites +)
    | Function  of (Function<double>*Term) // sin, cos, tan, ...
    | Connector of TermConnector

In [8]:
///**Description**
/// Schreibt eine eingerückte Liste der Termine in die Standardausgabe.
///
let rec printTerms (terms: Term list) tabCount =
    let tabPrefix = [0..tabCount] |> List.skip 1 |> List.map (fun _ -> "\t") |> List.fold (+) ""
    match terms with
    | [] -> 
        ()
    | (Function (f, term)) :: rest ->
        printfn "%s%s^%i" tabPrefix f.Name f.Power
        match term with
        | (SubTerm s) ->
            printTerms [term] tabCount
        | term ->
            printTerms [term] (tabCount + 1)
        printTerms rest tabCount
    | (SubTerm s) :: rest ->
        printTerms s (tabCount + 1)
        printTerms rest tabCount
    | head :: tail ->
        printfn "%s%A" tabPrefix head
        printTerms tail tabCount

In [9]:
///**Description**
/// Liste mit bekannte Ableitungen für Funktionen. Beziehen sich nicht auf das Argument,
/// welches gesondert durch die Kettenregel behandelt werden muss.
///
let createDerivedFunction (f: Function<double>) (argument: Term) =
    match f.Name with
    | "sin" ->
        [(Function (("cos" |> functionFromName), argument))]
    | "cos" ->
        [(Connector Subtraction); (Polynom (1, "<const>", 0));
        (Connector Multiplication) ; (Function (("sin" |> functionFromName), argument))]
    | _ ->
            failwith <| sprintf "Unknown function '%s'" f.Name
    (*[ 
        "sin", [(Connector Subtraction) ; (Polynom (1,"<const>",0)); (functionFromName "cos")];
        "cos", [(Connector Subtraction) ; (Polynom (1, "<const>",0))]
    ]*)

In [10]:
///**Description**
/// Interface welches den Vorgang des Aufteilens einer Entität und
/// der Aufteilung der einzelnen Teile in einen Teil welcher kontinuierlich das
/// Kriterium erfüllt und den Rest.
/// (Im Rest können also noch Teile sein welche das Kriterium erfüllen!)
///
type ISplitter<'TInput, 'TParts, 'TResult> =
    abstract member Splitter:   'TInput -> 'TParts list
    abstract member Criterium:  'TParts -> bool
    abstract member Aggregator: 'TParts list -> 'TResult
    
///**Description**
/// Definiert den Vorgang des Aufteilens einer Entität in einzelne Teile
/// welche solange aggregiert werden bis ein Teil das Kriterium nicht mehr
/// erfüllt.    
///
let splitBy (operations: ISplitter<'TInput, 'TParts, 'TResult>) input =
    let split     = input |> operations.Splitter
    //let passed    = (split |> List.head) :: (split |> List.skip 1 |> List.takeWhile operations.Criterium)
    let passed    = split |> List.takeWhile operations.Criterium
    let notPassed = split.[passed.Length..]
    (operations.Aggregator passed, operations.Aggregator notPassed)
    
///**Description**
/// Implementiert ISplitter für den Fall von Strings welche auf die bekannten Delimiter 
/// geprüft werden sollen.
///
let splitStringByDelimiter =
    splitBy 
        {
            new ISplitter<string, char, string> with
                member this.Splitter input   = [for c in input -> c]
                member this.Criterium c      = delimiters |> List.contains c |> not
                member this.Aggregator parts = System.String.Concat(parts)
        }
let splitStringGenerator (criterium: char -> bool) =
    splitBy
        {
            new ISplitter<char list, char, char list> with
                member this.Splitter input   = input
                member this.Criterium toTest = criterium toTest
                member this.Aggregator parts = parts
        }
        
let splitByTokenGenerator (criterium: Token -> bool) =
    splitBy
        {
            new ISplitter<Token list, Token, Token list> with
                member this.Splitter input   = input
                member this.Criterium toTest = criterium toTest
                member this.Aggregator parts = parts
        }

In [11]:
///**Description**
/// Nimmt so lange Elemente von einem String, bis der Char kein Buchstabe mehr ist.
///
let letterSplitter = splitStringGenerator (fun c -> Char.IsLetter(c))

///**Description**
/// Nimmt so lange Elemente von einem String, bis der Char keine Zahl mehr ist.
///
let numberSplitter = splitStringGenerator (fun c -> Char.IsNumber(c))

///**Description**
/// Wandelt die einzelnen Character in Strings um. Fasst dabei Zahlen, Identifier und Funktionen zusammen.
///
let parseChars (chars: char list) : (string list) =
    let rec parse (remaining: char list) (existing: string list): (string list) =
        match remaining with
        | [] -> existing
        | [single] -> (string single) :: existing
        | head :: tail ->
            match head with
            | letter when head |> isCharLetter -> 
                let matched, rest = letterSplitter tail
                ((head :: matched) |> System.String.Concat) :: (parse rest existing)
            | number when head |> isCharNumber -> 
                let matched, rest = numberSplitter tail
                ((head :: matched) |> System.String.Concat) :: (parse rest existing)
            | _ -> 
                (head |> string) :: (parse tail existing)            
    (parse chars []) 
    |> List.filter (fun s -> String.IsNullOrWhiteSpace(s) = false)

///**Description**
/// Prüft, ob ein String nur aus Zahlen besteht.
///
let isStringNumbersOnly s =
    [for c in s -> isCharNumber(c)] |> List.forall ((=) true)
    
///**Description**
/// Prüft, ob ein String nur aus Buchstaben besteht.
///
let isStringLettersOnly s =
    [for c in s -> isCharLetter(c)] |> List.forall ((=) true)

In [12]:
///**Description**
///
///
let convertStringListToTokenStream (parts: string list) =
    let tokens = parts |> List.map matchToToken |> TokenStreamContent
    tokens

///**Description**
/// Füllt die Liste mit Tokens mit impliziten Tokens auf.
/// Dazu wird zwischen jede Kombination aus Zahl, Id und funktion ein Multiplikationszeichen gesetzt
/// und zwischen jede Zahl/Id und öffnende Klammer ebeneso.
/// Z.B. wird 2x zu 2*x und -2sin(y) zu -2*sin(y)
///
let insertImplicitTokens (TokenStreamContent tokens) =
    
    let rec insert (notMatched: Token list) (matched: Token list) =
        match notMatched with
        | [] -> 
            matched
        // 4x -> 4*x
        | (Number n) :: (Id x) :: rest ->
            insert ((Id x) :: rest) (Times :: (Number n) :: matched)
        // 4sin -> 4*sin
        | (Number n) :: (Token.Function f) :: rest ->
            insert ((Token.Function f) :: rest) (Times :: (Number n) :: matched)
        // xsin -> x*sin
        | (Id x) :: (Token.Function f) :: rest ->
            insert ((Token.Function f) :: rest) (Times :: (Id x) :: matched)
        // 4( -> 4*(
        | (Number n) :: OpeningBracket :: rest ->
            insert (OpeningBracket :: rest) (Times :: (Number n) :: matched)
        // x( -> x*(
        | (Id x) :: OpeningBracket :: rest ->
            insert (OpeningBracket :: rest) (Times :: (Id x) :: matched)
        // sin( -> sin_(
        | (Token.Function f) :: OpeningBracket :: rest ->
            insert (OpeningBracket :: rest) (Token.Argument :: (Token.Function f) :: matched)
        // sin^2( -> sin^2_(
        | (Token.Function f) :: Hat :: (Number p) :: OpeningBracket :: rest ->
            insert (OpeningBracket :: rest) (Token.Argument :: (Number p) :: Hat :: (Token.Function f) :: matched)
        // )sin -> )*sin
        | ClosingBracket :: (Token.Function f) :: rest ->
            insert ((Token.Function f) :: rest) (Times :: ClosingBracket :: matched)
        // )x -> )*x
        | ClosingBracket :: (Id x) :: rest ->
            insert ((Id x) :: rest) (Times :: ClosingBracket :: matched)
        // )4 -> )*4
        | ClosingBracket :: (Number n) :: rest ->
            insert ((Number n) :: rest) (Times :: ClosingBracket :: matched)         
        // (x -> (+x
        | OpeningBracket :: (Id x) :: rest ->
            insert ((Id x) :: rest) (Plus :: OpeningBracket :: matched)
        // (4 -> (+4
        | OpeningBracket :: (Number n) :: rest ->
            insert ((Number n) :: rest) (Plus :: OpeningBracket :: matched)
        // (sin -> (+sin
        | OpeningBracket :: (Token.Function f) :: rest ->
            insert ((Token.Function f) :: rest) (Plus :: OpeningBracket :: matched)
        // _ -> _
        | head :: tail ->
            insert tail (head :: matched)
    
    let result = insert tokens [] 
                 |> List.rev
    
    // Die Gleichung muss auch mit einem Vorzeichen beginnen.
    match result.[0] with
    | Plus | Minus -> 
        (TokenStreamContent result)
    | _ ->
        (TokenStreamContent (Plus :: result))

In [13]:
///**Description**
/// Berechnet für jeden Token innerhalb des Streams wie tief in Klammern er verschachtelt ist.
///
let computeDepth (TokenStreamContent tokens) =
    let mutable depthCounter = 0

    let foldBracketCounter (foldedTokens, currentDepth) token =
        match token with
        | OpeningBracket ->
            ((token, currentDepth + 1) :: foldedTokens, currentDepth + 1)
        | ClosingBracket ->
            ((token, currentDepth) :: foldedTokens, currentDepth - 1)
        | _ ->
            ((token, currentDepth) :: foldedTokens, currentDepth)        
    
    let (result, _) = tokens |> List.fold foldBracketCounter ([], 0)
    result |> List.rev

In [14]:
///**Description**
/// Implementation des ISplitter-Interface welches solange Tokens liest bis ein passender
/// Token zum Schließen einer Klammer auf aktuellem Level gefunden wird.
///
let splitByClosingBracket =
    splitBy
        {
            new ISplitter<(Token*int) list, (Token*int), Token list> with
                member this.Splitter input   = [ for t in input -> t ]
                member this.Criterium toTest = toTest |> (fun (_, depth) -> depth <> 0) //toTest <> (_, depth)
                member this.Aggregator parts = parts  |> List.map (fun (t, _) -> t)
        }

In [15]:
// Bestimmt, ob ein Token ein Connector ist.
let isConnector (t: Token) =
    match t with
    | Plus | Minus | Times | Divided -> 
        true
    | _ -> 
        false

In [16]:
///**Description**
/// Fügt dem Ausdruck Klammern hinzu, so dass sich die Tiefe leichter bestimmen läßt.
///  2x^10sin(x- 5/3) cos(x^2) + 356x/10 -> 
/// (2x^10sin(x-(5/3))cos(x^2))+(356x/10)
///
let addImplicitBrackets (TokenStreamContent tokens) : TokenStream=
    let maxDepth = 
        (TokenStreamContent tokens) 
        |> computeDepth
        |> List.map (fun (_, depth) -> depth)
        |> List.max
        
    let rec insert (notMatched: Token list) (matched: Token list) =
        match notMatched with
        | [] -> matched
        | Plus  :: any :: ClosingBracket :: rest 
        | Plus  :: any :: Plus  :: rest 
        | Plus  :: any :: Minus :: rest
        | Minus :: any :: ClosingBracket :: rest
        | Minus :: any :: Plus  :: rest
        | Minus :: any :: Minus :: rest ->
            insert notMatched.[3..] ((notMatched.[0..2] |> List.rev) @ matched)
        | head :: tail ->
            insert tail (head :: matched)
        
    let withInserts = insert tokens [] |> List.rev
    (TokenStreamContent withInserts)

In [17]:
///**Description**
/// Wandelt Tokens in Terme um.
///
let rec matchTerms (TokenStreamContent tokens) : (Term list) =
    // TODO: warum die eine Liste mitschleppen?
    let rec matchTokensToTerms (unmatched: Token list) (matched: Term list) : (Term list) =
        if unmatched.Length = 0 then
            matched
        elif unmatched.Length = 1 then
            failwith (sprintf "Cannot match a single token. Two are required, connector and constant. Matched so far: %A, missing %A" matched unmatched)
        else
            let connector = match unmatched.[0] with
                            | Plus -> Addition
                            | Minus -> Subtraction
                            | Times -> Multiplication
                            | Divided -> Division
                            | _ -> failwith (sprintf "Connector is unknown: %A | next: %A" unmatched.[0] unmatched.[1..])

            // TODO: eventuell so umbauen, dass zwischen einem Hütchen und einer Zahl immer ein Plus
            //       eingebaut wird, sofern dort kein Minus steht
            match unmatched.[1..] with
            | [] -> matched
            // 2*x^10
            | (Number coefficient) :: Times :: (Id x) :: Hat :: (Number power) :: rest ->
                matchTokensToTerms rest ((Polynom (coefficient, x, power)) :: (Connector connector) :: matched)
            // 2*x^-10
            | (Number coefficient) :: Times :: (Id x) :: Hat :: Minus :: (Number power) :: rest ->
                matchTokensToTerms rest ((Polynom (coefficient, x, power * -1)) :: (Connector connector) :: matched)
            // 2*x
            | (Number coefficient) :: Times :: (Id x) :: rest ->
                matchTokensToTerms rest ((Polynom (coefficient, x, 1)) :: (Connector connector) :: matched)
            // 2
            | (Number constant) :: possibleConnector :: rest when (possibleConnector |> isConnector) || (possibleConnector = OpeningBracket) ->
                matchTokensToTerms (possibleConnector :: rest) ((Polynom (constant, "<const>", 0)) :: (Connector connector) :: matched)
            // x^10
            | (Id x) :: Hat :: (Number power) :: rest ->
                matchTokensToTerms rest ((Polynom (1, x, power)) :: (Connector connector) :: matched)
            // x^-10
            | (Id x) :: Hat :: Minus :: (Number power) :: rest ->
                matchTokensToTerms rest ((Polynom (1, x, power * -1)) :: (Connector connector) :: matched)
            // sin^2_(
            | (Token.Function f) :: Hat :: (Number n) :: Token.Argument :: OpeningBracket :: _ ->
                let withDepth = ((TokenStreamContent unmatched.[5..]) |> computeDepth)
                let argument, remaining = splitByClosingBracket withDepth
                let func = (Function ((functionFromNameAndPower f n), (SubTerm (matchTerms (TokenStreamContent argument.[1..argument.Length-2])))))
                matchTokensToTerms remaining (func :: (Connector connector) :: matched)
            // sin_(
            | (Token.Function f) :: Token.Argument :: OpeningBracket :: _ ->
                // Skip 2 more elements because they have a depth of 0.
                let withDepth = ((TokenStreamContent unmatched.[3..]) |> computeDepth)
                let argument, remaining = splitByClosingBracket withDepth
                let func = (Function ((functionFromName f), (SubTerm (matchTerms (TokenStreamContent argument.[1..argument.Length-2])))))
                matchTokensToTerms remaining (func :: (Connector connector) :: matched)
            // x
            | (Id x) :: possibleConnector :: rest when (possibleConnector |> isConnector) || (possibleConnector = OpeningBracket) ->
                matchTokensToTerms (possibleConnector :: rest) ((Polynom (1, x, 1)) :: (Connector connector) :: matched)
            // 2 (am Ende)
            | [(Number n)] ->
                ((Polynom (n, "<const>", 0)) :: (Connector connector) :: matched)
            // x (am Ende)
            | [(Id x)] ->
                ((Polynom (1, x, 1)) :: (Connector connector) :: matched)
            | rest -> 
                failwith (sprintf "Dont know how to match %A" rest)
    matchTokensToTerms tokens [] |> List.rev
    

In [18]:
///**Description**
/// Fasst aufeinanderfolgende Polynome mit gleicher Id zusammen.
///
(*
let rec aggregatePolynoms (terms: Term list) =
    match terms with
    | [] -> []
    | (Polynom (c1, id1, p1)) :: (Connector Multiplication) :: (Polynom (c2, id2, p2)) :: rest when id1 = id2 ->
        //printfn "mult coeff %i and %i" c1 c2
        //printfn "add power %i and %i" p1 p2
        (Polynom (c1*c2, id1, p1+p2)) :: aggregatePolynoms rest
    | head :: tail ->
        //printfn "head: %A || tail: %A" head tail
        head :: aggregatePolynoms tail
        
*)

In [19]:
///**Description**
/// Führt einige Vereinfachungen auf den Termen aus, z.B. Polynome zusammenfassen.
///
(*
let simplifyTerms (terms: Term list) =
    terms
    |> aggregateConnectors
    |> aggregatePolynoms
    |> replaceZeroPowerPolynomsWithConst
*)

## Weiterverarbeitung ##
Nachdem man die Gleichung in logische Komponenten aufgeteilt hat kann eine weitere Verarbeitung stattfinden. Im ersten Schritt wird dazu das Ableiten definiert. Aufgrund von komplexeren Termkombinationen ist es nötig die
    
 * Produktregeln
     \begin{equation*}
     \left( f(x)g(x) \right)^{\prime} = f^{\prime}(x)g(x) + f(x)g^{\prime}(x)
     \end{equation*}
 
 * Quotientenregel
     \begin{equation*}
     \left( \frac{f(x)}{g(x)} \right)^{\prime} = \frac{f^{\prime}(x)g(x) - f(x)g^{\prime}(x)}{g(x)^2}
     \end{equation*}
        
 * Kettenregel
     \begin{equation*}
     \left(f(g(x))\right)^{\prime} = g^{\prime}(x)\;f^{\prime}(g(x))
     \end{equation*}
        
zu berücksichtigen.

In [20]:
///**Description**
/// Leitet zwei Terme gemäß der Produktregel ab.
/// Wird als SubTerm zurückgegeben.
///
let rec deriveProduct (derive: Term->Term list) (a: Term) (b: Term list) : (Term) =
    match b with
    | [] -> (SubTerm [])
    | [single] ->
        let a' = derive a
        let b' = derive single
        [
            (derive a); [Connector Multiplication]; [single]; 
            [Connector Addition]; 
            [a]; [Connector Multiplication]; (derive single)
        ]
        |> List.concat |> SubTerm
    | head :: tail ->
        deriveProduct derive head tail

let defaultDeriver (t: Term) : (Term list) =
    failwith "not implemented"
    
let deriveDefaultProduct =
    deriveProduct defaultDeriver

///**Description**
/// Leitet zwei Terme gemäß der Quotientenregel ab.
///
let deriveQuotient (derive: Term->Term list) (a: Term) (b: Term) =
    printfn "Es wurde die Quotientenregel verwendet, dies sollte nicht passieren und Brüche sollten als negative Potenzen interpretiert werden."
    [
        (derive a); [Connector Multiplication ; Connector Subtraction]; 
        [a]; [Connector Multiplication]; (derive b); 
        [Connector Division]; 
        [SubTerm [b; b]]
    ]
    |> List.concat
(*
let deriveChain (derive: Term -> Term) (a: Term) (b: Term) =
    match a with
    | (Function (f, parameter)) ->
        derive parameter
    | _ ->
        derive a
    // TODO: so wohl nicht korrekt weil dann an anderer Stelle berücksichtig werden muss was reingeworfen wird
    //[(derive b); (Connector Multiplication); (derive a)]
*)

In [21]:
///**Description**
/// Teilt einen Ausdruck an einem Plus, oder Minuszeichen ohne in SubTerms zu steppen.
///
let splitTermByAdditionOrSubtraction (terms: Term list) = 
    
    let rec split (matched: Term list) (unmatched: Term list) =
        match unmatched with
        | [] -> 
            matched, unmatched
        |(Connector Addition) :: _ ->
            (matched, unmatched)
        | (Connector Subtraction) :: _ ->
            (matched, unmatched)
        | head :: tail  ->
            split (head :: matched) (tail)
    
    // Längenprüfung
    match terms.Length with
    | 0 ->
        ([],[])
    | 1 ->
        ([terms.[0]], [])
    | _ ->
        // der erste Term darf nicht als Trennzeichen verwendet werden,
        // muss aber im Ergebnis berücksichtigt werden
        let matching, rest = split [] terms.[1..]
        ((terms.[0] :: matching), rest)

///**Description**
/// Teilt eine Liste an Termen so auf, dass sie aus eigenständig
/// ableitbaren Termen besteht (d.h. getrennt durch Plus, oder Minus-Zeichen).
///
let rec splitIntoIndependentTermParts (terms: Term list) : (Term list list) =

    match splitTermByAdditionOrSubtraction (terms: Term list) with
    | ([], []) ->
        []
    | (matched, []) ->
        [matched]
    | (matched, unmatched) ->
        [matched] @ (unmatched |> splitIntoIndependentTermParts)

In [22]:
///**Description**
/// Wandelt einen Term in einen hübsch lesbaren String um.
///
let rec sprintTerm (t: Term) : string =
    
    match t with
        | (Connector c) ->
            match c with
            | Addition         -> "+"
            | Subtraction      -> "-"
            | Multiplication   -> "*"
            | Division         -> "/"
            | Argument         -> "o"
        | (Polynom p) ->
            match p with
            | (0, _, _)  -> ""
            | (n, "<const>", 1) -> sprintf "%i" n
            | (n, "<const>", e) -> sprintf "%i^%i" n e
            | (1, id, 1) -> id
            | (n, id, 1) -> sprintf "%i%s" n id
            | (n, id, p) -> sprintf "%i%s^%i" n id p
        | (Function (f, argument)) ->
            if f.Power <> 1 then
                sprintf "%s^%i(%s)" f.Name f.Power (sprintTerm argument)
            else
                sprintf "%s(%s)" f.Name (sprintTerm argument)
        | (SubTerm s) ->
            match s with
            | []           -> ""
            | [single]     -> single |> sprintTerm
            | head :: tail -> sprintf "%s%s" (head |> sprintTerm) ( (SubTerm tail) |> sprintTerm)

### Zusammenführung von Termen ###
Ersetzt sämtliche SubTerme aus einem Element gegen das Element an sich.

In [23]:
///**Description**
/// Ersetzt in einer Liste von Termen SubTerme mit einem einzelnen Element
/// gegen dieses Element.
///
let rec expandOneElementSubTerms (t: Term list) : (Term list) =    
    match t with
    | [] ->
        []
    | (SubTerm s) :: rest when s.Length = 1 -> 
        (s |> List.head) :: (expandOneElementSubTerms rest)
    | (SubTerm s) :: rest ->
        (expandOneElementSubTerms s) @ (expandOneElementSubTerms rest)
    | head :: tail ->
        head :: (expandOneElementSubTerms tail)
        
///**Description**
/// Fasst aufeinanderfolgende Rechenzeichen zusammen.
///
let rec aggregateConnectors (terms: Term list) : (Term list) =
    match terms with
    | [] -> []
    | (Connector Addition) :: (Connector Subtraction) :: rest ->
        (Connector Subtraction) :: (aggregateConnectors rest)
    | (Connector Subtraction) :: (Connector Addition) :: rest ->
        printfn "[aggregateConnectors] es wurde die folgende Kombination zum Vereinfachen gematched: - +, Absicht?"
        (Connector Subtraction) :: (aggregateConnectors rest)
    | (Connector Subtraction) :: (Connector Subtraction) :: rest ->
        (Connector Subtraction) :: (aggregateConnectors rest)
    | head :: tail ->
        head :: (aggregateConnectors tail)   
        
///**Description**
/// Ersetzt Polynome mit einer Potenz von null durch Konstanten.
///
let rec replaceZeroPowerPolynomsWithConst (terms: Term list) : (Term list) =
    match terms with
    | [] -> []
    | (Polynom (coef, id, 0)) :: rest when id <> "<const>" ->
        (Polynom (coef, "<const>", 0)) :: replaceZeroPowerPolynomsWithConst rest
    | head :: tail ->
        head :: replaceZeroPowerPolynomsWithConst tail

### Aggregation von Polynomen ###
Die folgenden Funktionen aggregieren Polynome:
 * Multiplikation zweier beliebiger Polynome
 * Division zweier beliebiger Polynome
 * Addition zweier Polynome mit gleicher Potenz und Id
 * Subtraktion zweier Polynome mit gleicher Potenz und Id

In [24]:
///**Description**
/// Führt eine Iteration über die Terme (Subterm!) durch und aggregiert dabei passende
/// Polynome. (Multiplikation/Division, Addition/Subtraktion)
///
let rec aggregatePolynoms (terms: Term list) : (Term list) =
    match terms with
    | [] ->
        []
    | [single] ->
        [single]
    | (Polynom (c1, id1, p1)) :: (Connector Multiplication) :: (Polynom (c2, id2, p2)) :: tail when id1 = id2 ->
        aggregatePolynoms (Polynom (c1*c2, id1, p1+p2) :: tail)
    | (Polynom (c1, id1, p1)) :: (Connector Division) :: (Polynom (c2, id2, p2)) :: tail when id1 = id2 ->
        aggregatePolynoms (Polynom (c1/c2, id1, p1-p2) :: tail)        
        //Polynom (c1*c2, id1, p1+p2) :: (aggregatePolynoms tail)
    | (Polynom (c1, id1, p1)) :: (Connector Addition) :: (Polynom (c2, id2, p2)) :: tail when id1 = id2 && p1 = p2 ->
        aggregatePolynoms (Polynom (c1+c2, id1, p1) :: tail)
    | (Polynom (c1, id1, p1)) :: (Connector Subtraction) :: (Polynom (c2, id2, p2)) :: tail when id1 = id2 && p1 = p2 ->
        aggregatePolynoms (Polynom (c1+c2, id1, p1) :: tail)
    | head :: tail ->
        head :: (aggregatePolynoms tail)

///**Description**
/// Führt eine Iteration von Polynomaggregation über den Term aus.
/// Im Falle eines SubTerm ist dies identisch zu aggregatePolynoms,
/// ansonsten wird der Term unverändert zurückgegeben.
///
let aggregatePolynomSubTerm (t: Term) : (Term) =
    match t with
    | SubTerm s ->
        SubTerm (aggregatePolynoms s)
    | any -> 
        any

### Ableiten der Terme ###
Aufgrund der aktuellen Typenklassifikation gibt es eine gesonderte Behandlung von Polynomen und Funktionen.
Besser wäre die Interpretation der Polynome als Funktionen.

In [29]:
///**Description**
/// Erzeugt zu einem Term eine Ableitung. Sollen mehrere Terme abgeleitet werden sind diese
/// in einem SubTerm zu wrappen.
///
let rec derive (depth: int) (term: Term) : (Term) =
    let id = generateId ()
    let indent = [0..depth] |> List.map (fun _ -> '\t') |> System.String.Concat
    
    printfn "%s>>>Deriving: %i" indent id
    printfn "%s%A" indent term
    
    let result = match term with
                 | (Connector c) ->
                     (Connector c)
                 | (Polynom (c, id, p)) ->
                     (Polynom (c*p, id, p - 1))
                 | (Function (f, argument)) ->
                     printfn "%sErstelle Ableitung für %A" indent term
                     let derivedArg = (derive (depth + 1) argument)
                     printfn "%sAbgeleitetes Argument: %A" indent derivedArg
                     let temp = (SubTerm [ derivedArg ; SubTerm (createDerivedFunction f argument)])
                     temp
                 | (SubTerm s) ->
                    match s with
                    | [] -> 
                        SubTerm []
                    | [single] ->
                        single |> derive (depth + 1)
                    | head :: tail ->
                        // TODO: Ohne einen Test der Connectoren geht hier gar nichts! :-O
                        SubTerm 
                            [
                                (derive (depth + 1) head) ; (Connector Multiplication) ; (SubTerm tail) ; 
                                (Connector Addition) ; 
                                (head) ; (Connector Multiplication) ; (derive (depth + 1) (SubTerm tail))
                            ]
    printfn "%s---Ergebnis: %i" indent id
    printfn "%s%A" indent result
    printfn "%s>>>End: %i" indent id
    result

### Tests ###
Zusammenstecken des ganzen Krams. Dieser Text dient hauptsächlich als optisches Trennelement.

In [30]:
//let sTest = simplifySubTerm (SubTerm [(Polynom (2,"x", 2)); (Connector Multiplication) ; (Polynom (3, "x", 3))]) 
//printfn "%s" (sprintTerm (SubTerm sTest))


/// Tests
///
let stringInput = "sin(x)"
//let input = [ for c in "2x^10*3x^2sin^2(x-5sin(x))cos(x^2)+356x/10-10cos(x+y)+(x+4)^2" -> c ]
//let input = [ for c in "2x^3*3x^5+x*sin(x)" -> c ]
let input = [ for c in stringInput -> c]
let split = parseChars input
let implicit = split 
               |> convertStringListToTokenStream 
               |> insertImplicitTokens 
               |> addImplicitBrackets

printfn "---Implicit:"
implicit |> printTokenStreamContent
printfn "---End"

let readyTerms = implicit |> matchTerms |> aggregatePolynoms

printfn "---Matched:"
printfn "%s" <| sprintTerm (SubTerm readyTerms)
printfn "---"

let splits = splitIntoIndependentTermParts readyTerms

printfn "---Splits:"
//printfn "%A" splits
splits |> List.iter (fun split -> printfn "%s" <| sprintTerm (SubTerm split))
printfn "---End"
//printfn "--------"
//readyTerms |> derive |> List.iter (fun x -> printfn "%A" x)

//splits |> List.iter (fun s -> printfn "%s%A" System.Environment.NewLine s)

let derived = splits |> List.map (fun split -> (SubTerm split) |> derive 0)
printfn "---Derived:"
printfn "%s" <| sprintTerm (SubTerm derived)
printfn "---End"

let aggregated = derived |> expandOneElementSubTerms 
let simplified = aggregated |> aggregatePolynoms

printfn ""
printfn "%A" derived
printfn ""

//let polynomTest = "2x^2*3x"
//let derivedPolynomProduct = 
//    (SubTerm [(Polynom (2, "x", 2));(Polynom (3, "x", 3))])
//    |> derive

//let expandedDerivedPolynomProduct =
//    [derivedPolynomProduct] |> expandOneElementSubTerms
    
//let simplified = (SubTerm expandedDerivedPolynomProduct) 
//                 |> aggregatePolynomSubTerm
                 
    //|> simplifySubTerm
(*
printfn "%A" derivedPolynomProduct
printfn "---------"
printfn "%A" expandedDerivedPolynomProduct
printfn "---------"
printfn "%A" simplified
*)
//(SubTerm derivedPolynomProduct)

//printfn "%s" <| sprintTerm (SubTerm derivedPolynomProduct)

//printfn "%A" (deriveDefaultProduct (Polynom (2, "x", 2) (Polynom(3, "x", 3))))
//splits |> List.iter (fun s -> printfn "%A%s" s System.Environment.NewLine)
//let derivedTerms = readyTerms |> derive

//printfn "%A" derivedTerms
//printfn "%A" (derive readyTerms)

//splits |> List.iter (fun s -> printfn "%A%s%s" s Environment.NewLine Environment.NewLine)
    


---Implicit:
+sin○(+x)
---End
---Matched:
+sin(+x)
---
---Splits:
+sin(+x)
---End
	>>>Deriving: 9
	SubTerm
  [Connector Addition;
   Function
     ({Name = "sin";
       Power = 1;
       Evaluate = <fun:knownFunctions@5>;},
      SubTerm [Connector Addition; Polynom (1, "x", 1)])]
		>>>Deriving: 10
		Connector Addition
		---Ergebnis: 10
		Connector Addition
		>>>End: 10
		>>>Deriving: 11
		SubTerm
  [Function
     ({Name = "sin";
       Power = 1;
       Evaluate = <fun:knownFunctions@5>;},
      SubTerm [Connector Addition; Polynom (1, "x", 1)])]
			>>>Deriving: 12
			Function
  ({Name = "sin";
    Power = 1;
    Evaluate = <fun:knownFunctions@5>;},
   SubTerm [Connector Addition; Polynom (1, "x", 1)])
			Erstelle Ableitung für Function
  ({Name = "sin";
    Power = 1;
    Evaluate = <fun:knownFunctions@5>;},
   SubTerm [Connector Addition; Polynom (1, "x", 1)])
				>>>Deriving: 13
				SubTerm [Connector Addition; Polynom (1, "x", 1)]
					>>>Deriving: 14
					Connector Addition
				